# Assignment 7

## Submit as an HTML file

### Print your name below

In [1]:
print("Lydia King")

Lydia King


### Import the "pandas" "numpy" and "statsmodels.formula.api" libraries

In [34]:
# Write your answer here:

import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

#### In the code chunk below read the CSV file named `results.csv` in the `data` <br> folder and print the first 5 rows of the dataset. Browse the dataset.

In [7]:
#read csv
results = pd.read_csv("data/results.csv")

#print the first 5 rows
print(results.head())

   resultId  raceId  driverId  constructorId number  grid position  \
0         1      18         1              1     22     1        1   
1         2      18         2              2      3     5        2   
2         3      18         3              3      7     7        3   
3         4      18         4              4      5    11        4   
4         5      18         5              1     23     3        5   

  positionText  positionOrder  points  laps         time milliseconds  \
0            1              1    10.0    58  1:34:50.616      5690616   
1            2              2     8.0    58       +5.478      5696094   
2            3              3     6.0    58       +8.163      5698779   
3            4              4     5.0    58      +17.181      5707797   
4            5              5     4.0    58      +18.014      5708630   

  fastestLap rank fastestLapTime fastestLapSpeed  statusId  
0         39    2       1:27.452         218.300         1  
1         41    3 

### (a)  Check Column Types and Data Cleaning

- Use the function .dtypes to get the column types
- Identify which columns have data types that might need conversion
- The 'milliseconds' column contains string values that should be numeric. Create a new column called 'race_time_ms' that:
    - Converts the column to a numeric data type
    - Replaces any non-numeric values with NaN

In [55]:
# Write your answer here

#find column types
results.dtypes

# If a column that's supposed to be "numeric" but appears as "object",
# then it needs to be cleaned and converted to numeric 

#We can see that number, position, positionText, positionOrder, time, milliseconds, fastest lap,
#rank, fastestlaptime, fastestlapspeed need to be converted


# I found the solution to removing NA's online - coerce identifies non-numeric numbers and automatically converts them
results["race_time_ms"] = pd.to_numeric(results["milliseconds"], errors ='coerce')


#OTHER METHOD


# figure out when milliseconds = numeric
is_numeric = results["milliseconds"].str.isnumeric()

# filter instances where it's not numeric
subset = results[is_numeric == False]


#make our lists
list_old = ['\\N']

list_new = [np.nan]

#convert to numeric and add in Nan

results["race_time_ms"] = results["milliseconds"].replace(list_old, list_new)
results["race_time_ms"] = pd.to_numeric(results["race_time_ms"])










### (b) Create Categorical Variables

- Create a new column called 'finish_category' that categorizes the race finish positions as follows:
    - Positions 1-3: 'Podium'
    - Positions 4-10: 'Points'
    - Positions 11-20: 'Midfield'
    - Positions >20: 'Backmarker'

Hint: Use the pd.cut() function

In [48]:
# Write your answer here

# Had to convert to numeric
results["position"] = pd.to_numeric(results["position"], errors='coerce')

# Define bins and labels
bins_x = [0, 3, 10, 20, float('inf')]
labels_x = ['Podium', 'Points', 'Midfield', 'Backmarker']

# Using the pd.cut function
results['finish_category'] = pd.cut(results["position"],
                                    bins=bins_x,
                                    labels=labels_x,
                                    right=True)

### (c) Calculate Race Duration
- For rows where 'milliseconds' is available, create a new column <br>
'race_duration_minutes' that converts milliseconds to minutes by dividing <br>
by (1000*60).
- Display the average race duration by 'constructorId' for the top 5 <br>
constructors with the shortest average race times

In [43]:
# Write your answer here

#New variable 
results["race_duration_minutes"] = results["race_time_ms"]/(1000*60)


# Sort
average_race_duration = (
    results.groupby("constructorId")
           .agg(mean_race_duration=('race_duration_minutes', 'mean'))
           .sort_values('mean_race_duration')  # Sort ascending to get shortest times
)

# Constructors with the top 5 average speeds
top_5_race = average_race_duration.head(5)

#Average of the top 5 speeds
average_top_5 = top_5_race["mean_race_duration"].mean()

#check work
print(average_top_5)

84.0898696372549


### (d) Driver Performance Analysis

- Calculate the following statistics for each driver, grouped by 'driverId':
    - Average finishing position
    - Total points
    - Number of races completed
    - Best finishing position

- Sort the results by total points in descending order
- Display the top 10 drivers based on total points

In [ ]:
# Write your answer here

# Grouping by driverId
driver_performance_analysis = (
    results.groupby("driverId")
           .agg( #note to self: only have to do .agg once
               mean_finishing_position=('positionOrder', 'mean'),
               total_points=('points', 'sum'), #what we use to get the sum
               races_completed=('raceId', 'count'),#just count number of races
               best_finishing_position=('positionOrder', 'min') #find min value
           )
           .sort_values('total_points', ascending=False)
)

# Show top 10 drivers by total points
print(driver_performance_analysis.head(10))

          mean_finishing_position  total_points  races_completed  \
driverId                                                           
1                        4.787097        4396.5              310   
20                       7.093333        3098.0              300   
4                        8.494413        2061.0              358   
830                      6.533742        1983.5              163   
8                        8.491477        1873.0              352   
822                      7.601990        1778.0              201   
3                        8.252427        1594.5              206   
30                       6.879870        1566.0              308   
817                      9.883621        1307.0              232   
18                       9.695793        1235.0              309   

          best_finishing_position  
driverId                           
1                               1  
20                              1  
4                               1  
830

### (e) Linear Regression
Create a linear regression model that predicts 'points' based on 'grid' (starting position) and 'laps' completed <br>
Use the following steps:

- Clean the data to remove any non-numeric values and missing values
- Create the regression formula using smf.ols 
- Display the summary of the regression model using model.summary()

What is the predicted points for a driver starting in position 3 and completing 55 laps?

Hint: Use ```.dropna()''' to remove missing values from the points, grid, and laps <br>
variables.

In [56]:
# Write your answer here

# Convert columns to numeric
results["points"] = pd.to_numeric(results["points"])
results["grid"] = pd.to_numeric(results["grid"])
results["laps"] = pd.to_numeric(results["laps"])

# Drop rows with missing values in the relevant columns
clean_data = results.dropna(subset=["points", "grid", "laps"])

# Create linear regression model
model = smf.ols(formula='points ~ grid + laps', data=clean_data).fit()

# Display summary of regression model
print(model.summary())

# Predict points for a driver starting in position 3 and completing 55 laps
predicted_points = model.predict({"grid": 3, "laps": 55})

print("Predicted points for when grid=3 and laps=55:", predicted_points.iloc[0])


                            OLS Regression Results                            
Dep. Variable:                 points   R-squared:                       0.215
Model:                            OLS   Adj. R-squared:                  0.215
Method:                 Least Squares   F-statistic:                     3530.
Date:                Mon, 24 Mar 2025   Prob (F-statistic):               0.00
Time:                        21:31:32   Log-Likelihood:                -70440.
No. Observations:               25840   AIC:                         1.409e+05
Df Residuals:                   25837   BIC:                         1.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.5841      0.054     48.267      0.0